In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

%matplotlib inline

tf.random.set_seed(34)
np.random.seed(34)

In [ ]:
df = pd.read_csv(r"../input/cassava-leaf-disease-classification/train.csv", dtype=str)

train, test = train_test_split(df, test_size = 0.1, random_state = 22, stratify = df['label'])

s = 256
t_datagen=ImageDataGenerator(preprocessing_function = tf.keras.applications.efficientnet.preprocess_input,
                          rotation_range=180,
                          width_shift_range=0.4,
                          height_shift_range=0.2,
                          zoom_range=0.4,
                          fill_mode='nearest',
                          horizontal_flip=True,
                          vertical_flip=True)

train_generator=t_datagen.flow_from_dataframe(
    dataframe=train,
    directory="../input/cassava-leaf-disease-classification/train_images",
    x_col="image_id",
    y_col="label",
    batch_size=32,
    shuffle=True,
    class_mode="categorical",
    target_size=(s,s))

v_datagen=ImageDataGenerator(preprocessing_function = tf.keras.applications.efficientnet.preprocess_input)

val_generator=v_datagen.flow_from_dataframe(
    dataframe=test,
    directory="../input/cassava-leaf-disease-classification/train_images",
    x_col="image_id",
    y_col="label",
    batch_size=32,
    shuffle=True,
    class_mode="categorical",
    target_size=(s,s))

In [ ]:
from keras.layers import Activation
from keras.utils.generic_utils import get_custom_objects
from keras import backend as k

def swish(x):
  return (k.sigmoid(x)*x)

get_custom_objects().update({'swish': Activation(swish)})

In [ ]:
weights = '../input/keras-pretrained-models/EfficientNetB7_NoTop_ImageNet.h5'
base = tf.keras.applications.EfficientNetB7(weights=weights,
                                            include_top=False,
                                            input_shape=(256,256,3),
                                            pooling='avg')


for layer in base.layers:
    layer.trainable=False

base.summary()

In [ ]:
x = tf.keras.layers.Dense(256, activation='swish')(base.output)
x = tf.keras.layers.Dropout(.2)(x)
x = tf.keras.layers.BatchNormalization()(x)
out = tf.keras.layers.Dense(5, activation='softmax')(x)

eff7model = tf.keras.models.Model(base.input, out)
eff7model.summary()

In [ ]:
eff7model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
            loss='categorical_crossentropy', 
            metrics=['accuracy'])

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=.2,
            mode='min',
            verbose=1,
            patience=2,
            min_delta=0.01)
            
ES = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            mode='min',
            verbose=1,
            patience=5,
            restore_best_weights=True,
            min_delta=0.01)

In [ ]:
history = eff7model.fit(train_generator,
                     steps_per_epoch=train_generator.n//train_generator.batch_size,
                     validation_data=val_generator,
                     validation_steps=val_generator.n//val_generator.batch_size,
                     epochs=30,
                     callbacks=[reduce_lr, ES])


In [ ]:
import os

TEST_DIR = '../input/cassava-leaf-disease-classification/test_images/'
test_images = os.listdir(TEST_DIR)
predictions = []

for image in test_images:
    img = tf.keras.preprocessing.image.load_img(TEST_DIR + image, target_size=(256,256))
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = tf.keras.applications.efficientnet.preprocess_input(img)
    pred = eff7model.predict(img).argmax(axis = 1)
    predictions.extend(pred)

labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
prediction = [labels[k] for k in predictions]

In [ ]:
sub = pd.DataFrame({'image_id': test_images, 'label': prediction})
display(sub)
sub.to_csv('submission.csv', index = False)